# Warehouse Data Generator

In [56]:
# This code is used to generate synthetic data for the SimioWarehouseSystem

In [57]:
# Input: a excel file which contains the parameter settings
#  \ Parameter              \ Type                \ Comment
#  \  # of orders           \  Integer            \
#  \  # of Skus             \  Integer            \
#  \  # of Locations        \  Integer            \
#  \  # of line per order   \  String             \ Distribution align with corresponding parameters
#  \  quantity per line     \  String             \ Distribution align with corresponding parameters
#  \  warehouse layout      \  Vector             \ Four corners coordinate value 

In [58]:
# OutPut: several csv files which contain the synthetic warehouse data
#    1. Orders.csv: 
#    2. Skus.csv:
#    3. Order_Skus.csv:
#    4. Locations.csv:
#    5. Skus_Locations.csv:

In [59]:
# import associated packages
from funcdefinations import *
import pandas as pd
import numpy as np
import random
import math
import datetime as dt
from datetime import datetime
from sklearn.utils import shuffle

## Get the paramters

In [60]:
# whether you want to use external file to pass parameters
External = False

if not External:
    # Set initial values (in case you don't want to use the xlsx file)
    Bounding     = [0, 0, 0, 1000, 1000, 1000, 1000, 0]
    # 0 - BLx; 1 - BLz; 2 - ULx, etc (going around clockwise - BL, UL, UR, BR)
    NumOrders    = 30
    NumSKUs      = 10 # Generate an error if the number of SKUs exceeds the number of locations
    SKUWeights   = [] # if empty, all equally likely.  If not empty, verify that the numbers work
    NumLocations = 24
    LineOrderDist = ['Uniform', 5, 5]
    QuantityLineDist = ['Uniform', 1,  5]
    PickCoopPair = pd.read_csv('Pick_Coop_pair.csv')
    PickerNum = 1
    TransporterNum = 2
else:
    [Bounding, NumOrders, NumSKUs, SKUWeights, NumLocations, LineOrderDist, QuantityLineDist] = readFromExcel(FileName = 'ParameterSetting.xlsx', SheetName = 'Parameter Setting', SKUWeightsSheet = 'SKU Weights')
    

## Get the dataframes and output the csv files

In [61]:
# Generate the initial dataframes
InitializationParaList = [NumOrders, NumSKUs, NumLocations, SKUWeights, LineOrderDist, QuantityLineDist]
[OrderInitial, SKUInitial, LocationInitial, OrderSKUInitial, SKULocationInitial]=dataframesInitialization(InitializationParaList)

# Complete the dataframes
# Complete the order dataframe
FinalOrderParaList = [OrderInitial]
OrderFinalDataframe = completeOrder(FinalOrderParaList)

# Complete the sku dataframe
FinalSKUParaList = [SKUInitial]
SKUFinalDataframe = completeSKU(FinalSKUParaList)

# Complete the location dataframe
FinalLocationParaList = [LocationInitial, Bounding]
LocationFinalDataframe = completeLocation(FinalLocationParaList)

# Complete the order_sku dataframe
FinalOrderSKUParaList = [OrderSKUInitial]
OrderSKUFinalDataframe = completeOrderSKU(FinalOrderSKUParaList)

# Complete the sku_location dataframe
FinalSKULocationParaList = [SKULocationInitial, PickCoopPair]
SKULocationFinalDataframe = completeSKULocation(FinalSKULocationParaList)

# Output csv files
# OutputCSVParaList = [OrderFinalDataframe,SKUFinalDataframe, LocationFinalDataframe, OrderSKUFinalDataframe, SKULocationFinalDataframe]
# outputCSV(OutputCSVParaList)

C:\Users\maxvi\OneDrive - Auburn University\Doctor work\Summer2020\Github\SimioWarehouseSystem\funcdefinations.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SkuID'][-Line:] =  sample(Population, SKUWeights, Line)


## Create schedule and route files

In [62]:
# create schedules and store them in OrderSkus dataframe
sc = CoopScheduler(PickerNum, TransporterNum)
OrderSKUFinalDataframe = sc.simple_scheduler(OrderFinalDataframe, OrderSKUFinalDataframe, 2)
# create route files for Simio
# create routes for pickers and transporters
sc.route_generator(OrderSKUFinalDataframe,SKULocationFinalDataframe)
OrderSKUFinalDataframe = sc.lineitem

C:\Users\maxvi\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [63]:
# create excel files for Simio
# write excel files
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Route.xlsx', engine='xlsxwriter')
# Write each route to a different worksheet.
for i in range(1, PickerNum+1):
    pd.DataFrame({'Route': sc.routes['route_p{}'.format(i)]}).to_excel(writer, sheet_name='Picker{}'.format(i),index = False)
for i in range(1, TransporterNum+1):
    pd.DataFrame({'Route': sc.routes['route_t{}'.format(i)]}).to_excel(writer, sheet_name='Transporter{}'.format(i),index = False)

writer.save()
# create excel files for Simio
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Order_Skus.xlsx', engine='xlsxwriter')
    
# Write each dataframe to a different worksheet.
OrderFinalDataframe.to_excel(writer,sheet_name = 'Orders',index=False)
SKULocationFinalDataframe.to_excel(writer,sheet_name = 'Skus',index=False)
OrderSKUFinalDataframe.to_excel(writer,sheet_name = 'LineItem',index=False)

writer.save()

In [64]:
df = sc.lineitem
df

,OrderID,SkuID,Quantity,PickTime,LoadTime,PickerID,TransporterID,BatchID,PickNodeID,CoopNodeID,frequency
11,3,8.0,2,60,30,1,1,1,4,2,1
12,3,10.0,1,60,30,1,1,1,6,3,1
16,4,10.0,2,60,30,1,2,1,6,3,1
4,1,4.0,3,60,30,1,1,1,11,6,1
3,1,9.0,3,60,30,1,1,1,16,8,1
...,...,...,...,...,...,...,...,...,...,...,...
148,30,3.0,2,60,30,1,2,8,23,12,1
146,30,2.0,4,60,30,1,2,8,2,1,2
147,30,6.0,2,60,30,1,2,8,1,1,2
142,29,7.0,2,60,30,1,1,8,20,10,2


In [65]:
mask = (df['TransporterID'] == 1) & (df['BatchID'] == 1)
df[mask]

,OrderID,SkuID,Quantity,PickTime,LoadTime,PickerID,TransporterID,BatchID,PickNodeID,CoopNodeID,frequency
11,3,8.0,2,60,30,1,1,1,4,2,1
12,3,10.0,1,60,30,1,1,1,6,3,1
4,1,4.0,3,60,30,1,1,1,11,6,1
3,1,9.0,3,60,30,1,1,1,16,8,1
13,3,1.0,5,60,30,1,1,1,17,9,1
1,1,7.0,5,60,30,1,1,1,20,10,2
2,1,5.0,3,60,30,1,1,1,19,10,2
0,1,6.0,4,60,30,1,1,1,1,1,3
10,3,2.0,4,60,30,1,1,1,2,1,3
14,3,6.0,4,60,30,1,1,1,1,1,3


In [66]:
mask2 = (df['PickerID'] == 2) & (df['BatchID'] == 1)
df[mask2]

,OrderID,SkuID,Quantity,PickTime,LoadTime,PickerID,TransporterID,BatchID,PickNodeID,CoopNodeID,frequency
